Check to see if jupyter lab uses the correct python interpreter with '!which python'.
It should be something like '/opt/anaconda3/envs/[environment name]/bin/python' (on Mac).
If not, try this: https://github.com/jupyter/notebook/issues/3146#issuecomment-352718675

In [1]:
import sys
sys.executable
#!which python #which does not semm to work on windows

'C:\\Users\\moritz\\anaconda3\\envs\\social-2\\python.exe'

# Install dependencies:

In [2]:
install_packages = True
if install_packages:
    !conda install tensorflow=2 -y
    !conda install -c anaconda pandas -y
    !conda install -c conda-forge html2text -y
    !conda install -c conda-forge tensorflow-hub -y # !conda install -c akode html2text -y
    !conda install -c conda-forge tqdm -y
    !conda install -c anaconda scikit-learn -y
    !conda install -c conda-forge matplotlib -y
    !conda install -c anaconda seaborn -y
    print("Done")

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\moritz\anaconda3\envs\social-2

  added / updated specs:
    - tensorflow=2


The following packages will be UPDATED:

  ca-certificates    anaconda::ca-certificates-2020.10.14-0 --> pkgs/main::ca-certificates-2021.1.19-haa95532_1
  certifi                anaconda::certifi-2020.6.20-py36_0 --> pkgs/main::certifi-2020.12.5-py36haa95532_0
  openssl               anaconda::openssl-1.1.1h-he774522_0 --> pkgs/main::openssl-1.1.1k-h2bbff1b_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\moritz\anaconda3\envs\social-2

  added / updated specs:
    - pandas


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2021.1.19-~ --> anaconda::ca-certificates-2020.10.1

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # conda install html2text


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\moritz\anaconda3\envs\social-2

  added / updated specs:
    - scikit-learn


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.12.~ --> anaconda::ca-certificates-2020.10.14-0
  certifi            conda-forge::certifi-2020.12.5-py36ha~ --> anaconda::certifi-2020.6.20-py36_0
  openssl            conda-forge::openssl-1.1.1k-h8ffe710_0 --> anaconda::openssl-1.1.1h-he774522_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\moritz\anaconda3\envs\social-2

  added / updated specs:
    - matplotlib


The following packages will be UPDATED:

  ca-certificates    anacond

# Imports

In [3]:
#imports
import pandas as pd
import numpy as np
import os
import time
import tensorflow as tf
import tensorflow_hub as hub
import zipfile
from html2text import HTML2Text
from tqdm import tqdm
import re
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import seaborn as sns

# Set pandas print options
This will improve readability of printed pandas dataframe.


In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Set global Parameters
Set your parameters here:

data_path: In this path put the data you have downloaded with YouTube Data Tools. 
output_path: Tghe files generated in this notebook will be saved here.

url_dict: URLs to models on Tensorflow hub are saved here. Other models are available there.
model_type: Define which model you would like to use. Choose one from url_dict

new_embeddings: If this is true, new embeddings will be generated and saved at output_path. Otherwise, embeddings are loaded from Disc.




In [5]:
path = os.path.abspath("..")
data_path = "./data/all_comments.csv"

output_path = "./output_w_usernames/"

print("input path: " + data_path)

new_embeddings = True

url_dict = {
            'Transformer' : "https://tfhub.dev/google/universal-sentence-encoder-large/5",
            'DAN' : "https://tfhub.dev/google/universal-sentence-encoder/4",
            'Transformer_Multilingual': "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3"
}

model_type = 'Transformer' #@param ['DAN','Transformer','Transformer_Multilingual']

input path: ./data/all_comments.csv


## Create output directory
Try to create the directory defined by output_path

In [6]:
try:
   os.mkdir(output_path)
  
except OSError:
  print ("Creation of the directory %s failed" % output_path)
else:
  print ("Successfully created the directory %s " % output_path)

try:
    os.mkdir(output_path + "cache/")
except OSError:
  print ("Creation of the directory %s failed" % output_path)
else:
  print ("Successfully created the directory %s " % output_path)


Creation of the directory ./output_w_usernames/ failed
Creation of the directory ./output_w_usernames/ failed


# Load Data
Load you data as a pandas dataframe

In [7]:
if new_embeddings:
    data = pd.read_csv(data_path, sep='\t',low_memory = False, header=(0))
    data.head()

# Preprocessing
Preprocess your data:
- Drop empty rows
- Drop unused columns

In [8]:
if new_embeddings:
    data = data.dropna(subset=['text', 'author_name']) # drop rows with no content
    data=data.drop(['id','likeCount'],axis=1)#, 'replyCount','authorChannelUrl','authorChannelId','isReplyTo','isReplyToName'] # drop unused columns
    data.head()

- remove HTML-tags, links and usernames

In [9]:
if new_embeddings:
    # Remove HTML tags
    tqdm.pandas()
    h = HTML2Text()
    h.ignore_links = True
    data['cleaned'] = data['text'].progress_apply(lambda x: h.handle(x))
    print( "Removed HTML Tags.")

    # Remove links
    http_link_pattern = r'http\S+'
    bitly_link_pattern = r'bit.ly/\S+'
    data['cleaned'] = data['cleaned'].str.replace(http_link_pattern, '')
    data['cleaned'] = data['cleaned'].str.replace(bitly_link_pattern, '')
    print( "Removed Links.")

    # Remove user names
    # keep_names = ["earth", "Tide", "Geologist", "A Person", "Titanic", "adventure", "Sun", "The United States Of America"] # user names we want to keep
    # user_names = [name for name in data['author_name'].unique() if (len(name)> 3 and name not in keep_names)]
    # data['cleaned'] = data['cleaned'].str.replace('|'.join(map(re.escape, user_names)), '')
    # print( "Removed user names.")

100%|██████████████████████████████████████████████████████████████████████| 1176916/1176916 [02:02<00:00, 9612.16it/s]


Removed HTML Tags.
Removed Links.


# Save or Load preprocessed data

Save your data afte preprocessing, or load preprocessed data from disc.

In [10]:
if new_embeddings:
  data.to_pickle(output_path+'data_preprocessed'+'.pkl')
else:
  data = pd.read_pickle(output_path+'data_preprocessed'+'.pkl')
data.head()

,Unnamed: 0,replyCount,published_at,author_name,text,authorChannelId,is_reply,threadId,video_id,video_published_at,search_term,cleaned
0,0,0.0,2021-02-12T08:03:13Z,cruhg,i suppose the rest of this amazing skit was banned off youtube,UCuaPGilCg2ex-mm1FekefcA,0.0,Ugw_Q_5TEdKb0aFMls94AaABAg,golAjKMDuVk,2019-05-22T15:29:45Z,climate change greta,i suppose the rest of this amazing skit was banned off youtube\n\n
1,1,0.0,2021-02-10T07:19:12Z,Fletcher Daniels,Watch the full version. It’s so much better.,UCJnQ8E1C5Mqr2-ZfBezxEOw,0.0,UgzcdsGz9l2lLDK5wPF4AaABAg,golAjKMDuVk,2019-05-22T15:29:45Z,climate change greta,Watch the full version. It’s so much better.\n\n
2,2,0.0,2021-02-01T03:38:05Z,rhymeswithteeth,"This is only a portion of the entire Greta Thunberg sketch. Where's the rest, GWPF?",UC1xAbwWReoezgoF1oMGVdMA,0.0,UgwdASJmWZyTKX9DJ4F4AaABAg,golAjKMDuVk,2019-05-22T15:29:45Z,climate change greta,"This is only a portion of the entire Greta Thunberg sketch. Where's the rest,\nGWPF?\n\n"
3,3,1.0,2021-01-30T17:57:20Z,rhymeswithteeth,"Interestingly, you can't find this comedy routine on Comedy Unleashed's YouTube page. It's been scrubbed off the page. I guess Comedy Unleashed's slogan, ""Free Thinking Comedy Club. No Self Censorship. If It's Funny, It's Funny"", doesn't apply to Will Franken's routine...too edgy for them, even. Seems supremely hypocritical.",UC1xAbwWReoezgoF1oMGVdMA,0.0,UgxhycLINnjG3mXlc2Z4AaABAg,golAjKMDuVk,2019-05-22T15:29:45Z,climate change greta,"Interestingly, you can't find this comedy routine on Comedy Unleashed's\nYouTube page. It's been scrubbed off the page. I guess Comedy Unleashed's\nslogan, ""Free Thinking Comedy Club. No Self Censorship. If It's Funny, It's\nFunny"", doesn't apply to Will Franken's routine...too edgy for them, even.\nSeems supremely hypocritical.\n\n"
4,4,0.0,2021-02-13T14:12:15Z,Matty Groves,Yep. He was cancelled by Comedy Unleashed for using his free speech to criticize them.,UCPnkqq5hKL8-bAKexApHv7w,1.0,UgxhycLINnjG3mXlc2Z4AaABAg,golAjKMDuVk,2019-05-22T15:29:45Z,climate change greta,Yep. He was cancelled by Comedy Unleashed for using his free speech to\ncriticize them.\n\n


# Produce Text Embeddings with Universal Sentence Encoder

## Load Model
Load the model from TF-hub

In [11]:
hub_url = url_dict[model_type]

os.environ['TFHUB_CACHE_DIR'] = output_path + "cache/"

if new_embeddings:
  print("Loading model. This will take some time...")
  embed = hub.load(hub_url)

Loading model. This will take some time...


## Embed Documents
Produce embeddings of your documents.

In [12]:
if new_embeddings:
    for k,g in data.groupby(np.arange(len(data))//200):
        if k == 0:
            embeddings = embed(g['cleaned'])
        else:
            embeddings_new = embed(g['cleaned'])
            embeddings = tf.concat(values=[embeddings,embeddings_new],axis = 0)
            print(k , end =" ")
    print("The embeddings vector is of fixed length {}".format(embeddings.shape[1]))

    np.save(output_path+'/embeddings'+model_type+'.npy', embeddings, allow_pickle=True, fix_imports=True)
else:
    embeddings = np.load(output_path+'/embeddings'+model_type+'.npy', mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

embeddings.shape

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

ResourceExhaustedError:  OOM when allocating tensor with shape[200,8,3622,3622] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node StatefulPartitionedCall/StatefulPartitionedCall/EncoderTransformer/Transformer/SparseTransformerEncode/Layer_0/SelfAttention/SparseMultiheadAttention/DotProductAttention/MatMul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_restored_function_body_82309]

Function call stack:
restored_function_body


## Calculate Similarity Matrix with angular distance

'Following Cer et al. (2018), we first compute
the sentence embeddings u, v for an STS sentence
pair, and then score the sentence pair similarity
based on the angular distance between the two
embedding vectors  d = − arccos (uv/||u|| ||v||).'

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def cos_sim(input_vectors):
    similarity = cosine_similarity(input_vectors)
    return similarity
cosine_similarity_matrix = cos_sim(np.array(embeddings))
print(cosine_similarity_matrix)

# Plots Similarity 
Plot and print a heat map showing the semantic contextual similarity between comments.

In [ ]:
import seaborn as sns
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  sns.set(font_scale=1.2)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")

num_samples = 5
off_set = 170

plot_similarity(data.iloc[off_set:off_set+num_samples]['cleaned'], embeddings[off_set:off_set+num_samples], 90)

# Show neighbours of a comment 
Define which comment to analyze

In [ ]:
comment_index = 171
comment = data["cleaned"][comment_index]
comment_list = data["cleaned"].tolist()
print(comment)

Print similar comments.

In [ ]:
def get_top_similar(sentence, sentence_list, similarity_matrix, topN):
    # find the index of sentence in list
    index = sentence_list.index(sentence)
    # get the corresponding row in similarity matrix
    similarity_row = np.array(similarity_matrix[index, :])
    # get the indices of top similar
    indices = similarity_row.argsort()[-topN:][::-1]
    return [sentence_list[i] for i in indices]


for i, value in enumerate(get_top_similar(comment, comment_list, cosine_similarity_matrix, 20)):
  print("Top similar comment {}: {}".format(i+1, value))